---
# **LAB 5 - Unrolling e parallelismo dinamico**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## [GPU Compute Capability](https://developer.nvidia.com/cuda-gpus)

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

## Bash and data setup

In [ ]:
#@title Bash setup
%%writefile /root/.bashrc

# If not running interactively, don't do anything
[ -z "$PS1" ] && return

# don't put duplicate lines in the history. See bash(1) for more options
# ... or force ignoredups and ignorespace
HISTCONTROL=ignoredups:ignorespace

# append to the history file, don't overwrite it
shopt -s histappend

# for setting history length see HISTSIZE and HISTFILESIZE in bash(1)
HISTSIZE=10000
HISTFILESIZE=20000

# check the window size after each command and, if necessary,
# update the values of LINES and COLUMNS.
shopt -s checkwinsize

# make less more friendly for non-text input files, see lesspipe(1)
[ -x /usr/bin/lesspipe ] && eval "$(SHELL=/bin/sh lesspipe)"

PS1='\[\033[01;34m\]\w\[\033[00m\]\$ '

# enable color support of ls and also add handy aliases
if [ -x /usr/bin/dircolors ]; then
    test -r ~/.dircolors && eval "$(dircolors -b ~/.dircolors)" || eval "$(dircolors -b)"
    alias ls='ls --color=auto'
    #alias dir='dir --color=auto'
    #alias vdir='vdir --color=auto'

    alias grep='grep --color=auto'
    alias fgrep='fgrep --color=auto'
    alias egrep='egrep --color=auto'
fi

# some more ls aliases
alias ll='ls -lF'
alias la='ls -A'
alias l='ls -CF'

# path setup
export PATH="./:/usr/local/cuda/bin:$PATH"

In [ ]:
!source /root/.bashrc

Clone GPUcomputing site on github...

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

Define some paths...

In [ ]:
# path setup
!mkdir -p /content/GPUcomputing/lab5
%cd /content/GPUcomputing/lab5
!mkdir -p device
!mkdir -p dynParall
!mkdir -p unrolling
!mkdir -p MQDB-CUDA-DP

# ▶️ VS Code on Colab

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "gpu" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

# Optional: if you want to clone a Github or Gitlab repository
repository_url="https://github.com/giulianogrossi/GPUcomputing" #@param {type: "string"}
init_git_cloudflared(repository_url)

# ✅ DeviceQuery

In [ ]:
%%writefile device/helper.h
// Beginning of GPU Architecture definitions
inline int _ConvertSMVer2Cores(int major, int minor) {
	// Defines for GPU Architecture types (using the SM version to determine
	// the # of cores per SM
	typedef struct {
		int SM;  // 0xMm (hexidecimal notation), M = SM Major version,
		// and m = SM minor version
		int Cores;
	} sSMtoCores;

	sSMtoCores nGpuArchCoresPerSM[] = {
			{0x20, 32},
			{0x30, 192},
			{0x32, 192},
			{0x35, 192},
			{0x37, 192},
			{0x50, 128},
			{0x52, 128},
			{0x53, 128},
			{0x60,  64},
			{0x61, 128},
			{0x62, 128},
			{0x70,  64},
			{0x72,  64},
			{0x75,  64},
			{0x80,  64},
      {0x86, 128},
      {0x87, 128},
			{-1, -1}};

	int index = 0;

	while (nGpuArchCoresPerSM[index].SM != -1) {
		if (nGpuArchCoresPerSM[index].SM == ((major << 4) + minor)) {
			return nGpuArchCoresPerSM[index].Cores;
		}

		index++;
	}

	//# If we don't find the values, we default use the previous one to run properly
	printf(
			"MapSMtoCores for SM %d.%d is undefined."
			"  Default to use %d Cores/SM\n",
			major, minor, nGpuArchCoresPerSM[index - 1].Cores);
	return nGpuArchCoresPerSM[index - 1].Cores;
}

inline const char* _ConvertSMVer2ArchName(int major, int minor) {
  // Defines for GPU Architecture types (using the SM version to determine
  // the GPU Arch name)
  typedef struct {
    int SM;  // 0xMm (hexidecimal notation), M = SM Major version,
    // and m = SM minor version
    const char* name;
  } sSMtoArchName;

  sSMtoArchName nGpuArchNameSM[] = {
      {0x30, "Kepler"},
      {0x32, "Kepler"},
      {0x35, "Kepler"},
      {0x37, "Kepler"},
      {0x50, "Maxwell"},
      {0x52, "Maxwell"},
      {0x53, "Maxwell"},
      {0x60, "Pascal"},
      {0x61, "Pascal"},
      {0x62, "Pascal"},
      {0x70, "Volta"},
      {0x72, "Xavier"},
      {0x75, "Turing"},
      {0x80, "Ampere"},
      {0x86, "Ampere"},
      {-1, "Graphics Device"}};

  int index = 0;

  while (nGpuArchNameSM[index].SM != -1) {
    if (nGpuArchNameSM[index].SM == ((major << 4) + minor)) {
      return nGpuArchNameSM[index].name;
    }
    index++;
  }

  // If we don't find the values, we default use the previous one
  // to run properly
  printf(
      "MapSMtoArchName for SM %d.%d is undefined."
      "  Default to use %s\n",
      major, minor, nGpuArchNameSM[index - 1].name);
  return nGpuArchNameSM[index - 1].name;
}
// end of GPU Architecture definitions


In [ ]:
%%writefile device/deviceQuery.cu
#include <stdlib.h>
#include <stdio.h>
#include "helper.h"
#include "../../utils/common.h"

int main(void) {

	printf("\nCUDA Device Query (Runtime API) version (CUDART static linking)\n\n");
	int deviceCount = 0;
	CHECK(cudaGetDeviceCount(&deviceCount));

	// This function call returns 0 if there are no CUDA capable devices.
	if (deviceCount == 0)
		printf("There are no available device(s) that support CUDA\n");
	else
		printf("Detected %d CUDA Capable device(s)\n", deviceCount);

	int dev, driverVersion = 0, runtimeVersion = 0;

	for (dev = 0; dev < deviceCount; ++dev) {
		cudaSetDevice(dev);
		cudaDeviceProp deviceProp;
		cudaGetDeviceProperties(&deviceProp, dev);

		printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);

		cudaDriverGetVersion(&driverVersion);
		cudaRuntimeGetVersion(&runtimeVersion);

		printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
				driverVersion / 1000, (driverVersion % 100) / 10,
				runtimeVersion / 1000, (runtimeVersion % 100) / 10);

		printf("  GPU arch name:                                 %s\n",
						_ConvertSMVer2ArchName(deviceProp.major, deviceProp.minor));

		printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
				deviceProp.major, deviceProp.minor);

		printf("  Total amount of global memory:                 %.0f MBytes (%llu bytes)\n",
				(float) deviceProp.totalGlobalMem / 1048576.0f,
				(unsigned long long) deviceProp.totalGlobalMem);

		printf("  (%2d) Multiprocessors, (%3d) CUDA Cores/MP:     %d CUDA Cores\n",
						deviceProp.multiProcessorCount,
						_ConvertSMVer2Cores(deviceProp.major, deviceProp.minor),
						_ConvertSMVer2Cores(deviceProp.major, deviceProp.minor) *
								deviceProp.multiProcessorCount);
		
		printf("  GPU Max Clock rate:                            %.0f MHz (%0.2f GHz)\n",
				deviceProp.clockRate * 1e-3f, deviceProp.clockRate * 1e-6f);

		printf("  Memory Clock rate:                             %.0f Mhz\n", deviceProp.memoryClockRate * 1e-3f);
		printf("  Memory Bus Width:                              %d-bit\n", deviceProp.memoryBusWidth);
		if (deviceProp.l2CacheSize)
			printf("  L2 Cache Size:                                 %d bytes\n", deviceProp.l2CacheSize);

		printf("  Maximum Texture Dimension Size (x,y,z)         1D=(%d), 2D=(%d, %d), 3D=(%d, %d, %d)\n",
				deviceProp.maxTexture1D, deviceProp.maxTexture2D[0],
				deviceProp.maxTexture2D[1], deviceProp.maxTexture3D[0],
				deviceProp.maxTexture3D[1], deviceProp.maxTexture3D[2]);

		printf("  Maximum Layered 1D Texture Size, (num) layers  1D=(%d), %d layers\n",
				deviceProp.maxTexture1DLayered[0],
				deviceProp.maxTexture1DLayered[1]);

		printf("  Maximum Layered 2D Texture Size, (num) layers  2D=(%d, %d), %d layers\n",
				deviceProp.maxTexture2DLayered[0],
				deviceProp.maxTexture2DLayered[1],
				deviceProp.maxTexture2DLayered[2]);

		printf("  Total amount of constant memory                %lu bytes\n",
				deviceProp.totalConstMem);
		printf("  Total amount of shared memory per block        %lu bytes\n",
				deviceProp.sharedMemPerBlock);
		printf("  Total number of registers available per block  %d\n",
				deviceProp.regsPerBlock);
		printf("  Warp size                                      %d\n",
				deviceProp.warpSize);
		printf("  Maximum number of threads per multiprocessor   %d\n",
				deviceProp.maxThreadsPerMultiProcessor);
		printf("  Maximum number of threads per block            %d\n",
				deviceProp.maxThreadsPerBlock);
		printf("  Max dimension size of a thread block (x,y,z)  (%d, %d, %d)\n",
				deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1],
				deviceProp.maxThreadsDim[2]);
		printf("  Max dimension size of a grid size    (x,y,z)  (%d, %d, %d)\n",
				deviceProp.maxGridSize[0], deviceProp.maxGridSize[1],
				deviceProp.maxGridSize[2]);
		printf("  Maximum memory pitch                           %lu bytes\n",
				deviceProp.memPitch);
		printf("  Texture alignment                              %lu bytes\n",
				deviceProp.textureAlignment);
		printf("  Concurrent copy and kernel execution           %s with %d copy engine(s)\n",
				(deviceProp.deviceOverlap ? "Yes" : "No"),
				deviceProp.asyncEngineCount);
		printf("  Run time limit on kernels                      %s\n",
				deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
		printf("  Integrated GPU sharing Host Memory             %s\n",
				deviceProp.integrated ? "Yes" : "No");
		printf("  Support host page-locked memory mapping        %s\n",
				deviceProp.canMapHostMemory ? "Yes" : "No");
		printf("  Alignment requirement for Surfaces             %s\n",
				deviceProp.surfaceAlignment ? "Yes" : "No");
		printf("  Device has ECC support                         %s\n",
				deviceProp.ECCEnabled ? "Enabled" : "Disabled");

		printf("  Device supports Unified Addressing (UVA):      %s\n",
				deviceProp.unifiedAddressing ? "Yes" : "No");
		printf("  Device PCI Domain ID / Bus ID / location ID:   %d / %d / %d\n",
				deviceProp.pciDomainID, deviceProp.pciBusID,
				deviceProp.pciDeviceID);
	}
	return 0;
}


In [ ]:
# Compilazione ed esecuzione

!nvcc device/deviceQuery.cu -o deviceQuery
!./deviceQuery

# ✅ Unrolling


In [ ]:
%%writefile unrolling/main.cu
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#include "../../utils/common.h"

// the kernels prototype
__global__ void blockParReduceUroll(int*, int*, ulong);
__global__ void multBlockParReduceUroll8(int*, int*, ulong); 
__global__ void multBlockParReduceUroll16(int*, int*, ulong);


/*
 *  Block by block parallel implementation with divergence
 */
__global__ void blockParReduce1(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 *  Block by block parallel implementation without divergence
 */
__global__ void blockParReduce2(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1)  {
		if (tid < stride)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 * MAIN: test on parallel reduction
 */
int main(void) {
	int *a, *b, *d_a, *d_b;
	int blockSize = 1024;            // block dim 1D
	ulong numBlock = 1024*1024;      // grid dim 1D
	ulong n = blockSize * numBlock;  // array dim
	long sum_CPU = 0, sum_GPU;
	long nByte = n*sizeof(int), mByte = numBlock * sizeof(int);
	double start, stopGPU, stopCPU, speedup;

	printf("\n****  test on parallel reduction  ****\n");

	// init
	a = (int *) malloc(nByte);
	b = (int *) malloc(mByte);
	CHECK(cudaMalloc((void **) &d_a, nByte));
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void **) &d_b, mByte));
	CHECK(cudaMemset((void *) d_b, 0, mByte));

	/***********************************************************/
	/*                     CPU reduction                       */
	/***********************************************************/
	printf("  Vector length: %.2f MB\n",n/(1024.0*1024.0));
	printf("\n  CPU procedure...\n");
	start = seconds();
	for (ulong i = 0; i < n; i++) sum_CPU += a[i];
	stopCPU = seconds() - start;
	printf("    Elapsed time: %f (sec) \n", stopCPU);
	printf("    sum: %lu\n",sum_CPU);

	printf("\n  GPU kernels (mem required %lu bytes)\n", nByte);

	/***********************************************************/
	/*         KERNEL blockParReduce1 (divergent)              */
	/***********************************************************/
	// block by block parallel implementation with divergence
	printf("\n  Launch kernel: blockParReduce1...\n");
	start = seconds();
	blockParReduce1<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaGetLastError());
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);
	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i]=1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*        KERNEL blockParReduce2  (non divergent)          */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduce2...\n");
	start = seconds();
	blockParReduce2<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
//		printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);
	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*               KERNEL blockParReduceUroll                */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduceUroll...\n");
	start = seconds();
	blockParReduceUroll<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);
	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*            KERNEL multBlockParReduceUroll8              */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: multBlockParReduceUroll8...\n");
	start = seconds();
	multBlockParReduceUroll8<<<numBlock/8, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock/8; i++)
		sum_GPU += b[i];
	printf("    sum: %lu\n",sum_GPU);
	assert(sum_GPU == n);
	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*            KERNEL multBlockParReduceUroll16             */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: multBlockParReduceUroll16...\n");
	start = seconds();
	multBlockParReduceUroll16<<<numBlock/16, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock/16; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	cudaFree(d_a);

	CHECK(cudaDeviceReset());
	return 0;
}



# 🔴 TODO

In [ ]:
%%writefile unrolling/preduceUnroll.cu
#include <stdio.h>

/*
 *  Device function: block parallel reduction based on warp unrolling
 */
__device__ void blockWarpUnroll(int *thisBlock, int blockDim, uint tid) {
  // in-place reduction in global memory
    for (int stride = blockDim / 2; stride > 32; stride >>= 1)  {
      if (tid < stride)
        thisBlock[tid] += thisBlock[tid + stride];

      // synchronize within threadblock
      __syncthreads();
    }

    // unrolling warp
    if (tid < 32) {
      volatile int *vmem = thisBlock;
      vmem[tid] += vmem[tid + 32];
      vmem[tid] += vmem[tid + 16];
      vmem[tid] += vmem[tid + 8];
      vmem[tid] += vmem[tid + 4];
      vmem[tid] += vmem[tid + 2];
      vmem[tid] += vmem[tid + 1];
    }
}

/*
 *  Block by block parallel implementation with warp unrolling
 */
__global__ void blockParReduceUroll(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

  // block parall. reduction based on warp unrolling 
  blockWarpUnroll(thisBlock, blockDim.x, tid);

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 *  Multi block parallel implementation with block and warp unrolling
 */
__global__ void multBlockParReduceUroll8(int *in, int *out, ulong n) {

	 
	 // TODO

}

/*
 *  Multi block parallel implementation with block and warp unrolling
 */
__global__ void multBlockParReduceUroll16(int *in, int *out, ulong n) {

	// TODO
	
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_37 unrolling/main.cu unrolling/preduceUnroll.cu   -o preduceUnroll
!./preduceUnroll

# ✅ Parallelismo dinamico

In [ ]:
%%writefile MQDB-CUDA-DP/main.cu

#include "../../utils/MQDB/mqdb.h"
#include "../../utils/common.h"

#define BLOCK_SIZE 16     // block size

struct tms {
	double GPUtmsMQDB;
	double GPUtmsMQDBDynPar1;
	double GPUtmsMQDBDynPark;
	float density;
};

// the kernels prototype
__global__ void mqdbBlockProd(mqdb, mqdb, mqdb, uint, uint, uint);
__global__ void mqdbProdDP1(mqdb, mqdb, mqdb, uint, uint); 
__global__ void mqdbProdDPk(mqdb, mqdb, mqdb, uint); 

/*
 * Test on MQDB kernels
 */
void testKernelsMQDB(uint n, uint k, struct tms* times) {

	// mqdb host matrices
	mqdb A, B, C, C1;

	// mqdb device matrices
	mqdb d_A, d_B, d_C;

	// fill in
	A = mqdbConst(n, k, 10, 1);
	B = mqdbConst(n, k, 10, 1);
	C = mqdbConst(n, k, 10, 1);
	C1 = mqdbConst(n, k, 10, 1);

	ulong nBytes = n * n * sizeof(float);
	ulong kBytes = k * sizeof(uint);
	printf("Memory size required = %.1f (MB)\n",(float)nBytes/(1024.0*1024.0));

	// malloc and copy on device memory
	d_A.nBlocks = A.nBlocks;
	CHECK(cudaMalloc((void**)&d_A.blkSize, kBytes));
	CHECK(cudaMemcpy(d_A.blkSize, A.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_A.elem, nBytes));
	CHECK(cudaMemcpy(d_A.elem, A.elem, nBytes, cudaMemcpyHostToDevice));
	d_B.nBlocks = B.nBlocks;
	CHECK(cudaMalloc((void**)&d_B.blkSize, kBytes));
	CHECK(cudaMemcpy(d_B.blkSize, B.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_B.elem, nBytes));
	CHECK(cudaMemcpy(d_B.elem, B.elem, nBytes, cudaMemcpyHostToDevice));
	d_C.nBlocks = C.nBlocks;
	CHECK(cudaMalloc((void**)&d_C.blkSize, kBytes));
	CHECK(cudaMemcpy(d_C.blkSize, C.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_C.elem, nBytes));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*                     GPU MQDB product                    */
	/***********************************************************/
	printf("Kernel MQDB product...\n");
	dim3 block(BLOCK_SIZE, BLOCK_SIZE);
	dim3 grid((n + block.x - 1) / block.x, (n + block.y - 1) / block.y);
	uint sdim = 0;
	double start = seconds();
	for (uint i = 0; i < k; i++ ) {
		uint d = A.blkSize[i];
		mqdbBlockProd<<<grid, block>>>(d_A, d_B, d_C, sdim, d, n);
		sdim += d;
	}
	CHECK(cudaDeviceSynchronize());
	double GPUtime2 = seconds() - start;
	printf("   elapsed time:                    %.2f (sec)\n", GPUtime2);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*              GPU MQDB dynamic par. GRID(1)              */
	/***********************************************************/
	start = seconds();
	printf("Kernel MQDB product with dynamic parall. GRID(1)...\n");
	mqdbProdDP1<<< 1, 1 >>>(d_A, d_B, d_C, k, n);
	CHECK(cudaDeviceSynchronize());
	double GPUtime3 = seconds() - start;
	printf("   elapsed time:                        %.2f (sec)\n", GPUtime3);
	printf("   speedup vs GPU MQDB product:         %.2f\n", GPUtime2/GPUtime3);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*              GPU MQDB dynamic par. GRID(k)              */
	/***********************************************************/
	start = seconds();
	printf("Kernel MQDB product with dynamic parall. GRID(k)...\n");
	mqdbProdDPk<<< 1, k >>>(d_A, d_B, d_C, n);
	CHECK(cudaDeviceSynchronize());
	double GPUtime4 = seconds() - start;
	printf("   elapsed time:                        %.2f (sec)\n", GPUtime4);
	printf("   speedup vs GPU MQDB product:         %.2f\n", GPUtime2/GPUtime4);
	printf("   speedup vs GPU MQDB product GRID(1): %.2f\n", GPUtime3/GPUtime4);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	CHECK(cudaFree(d_A.elem));
	CHECK(cudaFree(d_B.elem));
	CHECK(cudaFree(d_C.elem));

	// collect times
	times->GPUtmsMQDB = GPUtime2;
	times->GPUtmsMQDBDynPar1 = GPUtime3;
	times->GPUtmsMQDBDynPark = GPUtime4;
	float den = 0;
	for (uint j = 0; j < k; j++)
		den += A.blkSize[j]*A.blkSize[j];
	times->density = den/(n*n);
}

/*
 * main function
 */
int main(int argc, char *argv[]) {
	uint n = 16*1024;      // matrix size
	uint min_k = 10;       // max num of blocks
	uint max_k = 20;       // max num of blocks

	struct tms times[max_k-min_k+1];

	// multiple tests on kernels
	for (uint k = min_k; k <= max_k; k++) {
		printf("\n*****   k = %d --- (avg block size = %f)\n",k,(float)n/k);
		testKernelsMQDB(n, k, &times[k-min_k]);
	}

	FILE *fd;
	fd = fopen("res.csv", "w");
	if (fd == NULL) {
		perror("file error!\n");
		exit(1);
	}

	// write results on file
	fprintf(fd,"num blocks,");
		for (uint j = 0; j <= max_k-min_k; j++)
			fprintf(fd,"%d,",j+min_k);

	fprintf(fd,"\nKernel MQDB product,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDB);

	fprintf(fd,"\nKernel MQDB product with dynamic parall. GRID(1),");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDBDynPar1);

	fprintf(fd,"\nKernel MQDB product with dynamic parall. GRID(k),");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDBDynPark);

	fprintf(fd,"\ndensity,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].density);

	fclose(fd);

	return 0;
}




# 🔴 TODO

In [ ]:
%%writefile MQDB-CUDA-DP/mqdb_DP.cu

#include "../../utils/MQDB/mqdb.h"

#define BLOCK_SIZE 16     // block size


/*
 * Kernel for block sub-matrix product of mqdb
 */
__global__ void mqdbBlockProd(mqdb A, mqdb B, mqdb C, uint sdim, uint d, uint n) {
	int row = blockIdx.y * blockDim.y + threadIdx.y;
	int col = blockIdx.x * blockDim.x + threadIdx.x;

	// jump to the right block sub-matrix
	int  offset = (n+1)*sdim;

	// each thread computes an entry of the product matrix
	if ((row < d) && (col < d)) {
		float val = 0;

		for (int k = 0; k < d; k++)
			val += A.elem[row * n + k + offset] * B.elem[k * n + col + offset];
		C.elem[row * n + col + offset] = val;
	}
}

/*
 * Kernel for block sub-matrix product of mqdb: parent grid(1)
 */
__global__ void mqdbProdDP1(mqdb A, mqdb B, mqdb C, uint k, uint n) {
	
	// TODO
}

/*
 * Kernel for block sub-matrix product of mqdb: parent grid(k)
 */
__global__ void mqdbProdDPk(mqdb A, mqdb B, mqdb C, uint n) {
	
	// TODO

}


In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_37 -dc MQDB-CUDA-DP/mqdb_DP.cu MQDB-CUDA-DP/main.cu ../utils/MQDB/mqdb.cpp 
!nvcc -arch=sm_37 mqdb_DP.o main.o mqdb.o -o mqdb_prod
!rm -rf *.o
!./mqdb_prod